In [1]:
import os
import json
import pandas as pd
import numpy as np
import time
from tqdm import tqdm  # Import tqdm for the progress bar
from evaluator import evaluate
from eslint import EslintProcessor
from google_closure_compiler import GoogleClosureCompiler
from chatgpt_api import ChatGPT  # Import ChatGPT
from gemini_api import gemini_response,gemini_json_response # Import Gemini API
from collections import defaultdict
import enum
from typing_extensions import TypedDict

/home/metehan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
PROMPT_CODEGEN="""
Task: Generate JavaScript code based on the following description.

Problem Description: {leetcode_desc}

Give your output in this particular schema: {schema}

Requirements:
1. Use modern JavaScript syntax (ES6+)
2. Implement error handling where appropriate
3. Follow best practices for code organization and readability
"""

PROMPT_REFACTOR="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Problem Description: {leetcode_desc}

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
"""

PROMPT_REFACTOR_V2="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
"""


# Experiment 3: Refactor with metadata from LeetCode added
PROMPT_REFACTOR_V3="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Problem Description: {leetcode_desc}

Examples: {examples}

Constraint: {constraint}

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
"""

# Experiment 5: Refactor with more specific requirements (e.g., make the solution 10 lines or less) (not trivial, the above cases first)
PROMPT_REFACTOR_V5="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Problem Description: {leetcode_desc}

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
"""

out_schema = {
    "code": "generated javascript code",
    "rationale": "explain your approach keep it under 200 words"
}
gemini_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "code": {"type": "string"},
            "rationale": {"type": "string"}
        },
        "required": ["code", "rationale"]
    }
}



In [5]:
import pandas as pd

df = pd.read_csv('Code Generation Dataset NLP - Sheet1.csv')

print(df.head())
print(df.columns)

   Problem Number                                        Name  \
0               1                 Create Hello World Function   
1               2                                   Counter 2   
2               3  Apply Transform Over Each Element in Array   
3               4                  Filter Elements from Array   
4               5                 Array Reduce Transformation   

                                         Description  \
0  Write a function createHelloWorld. It should r...   
1  Write a function createCounter. It should acce...   
2  Given an integer array arr and a mapping funct...   
3  Given an integer array arr and a filtering fun...   
4  Given an integer array nums, a reducer functio...   

                                            Examples  \
0  Example 1:\n\nInput: args = []\nOutput: "Hello...   
1  Example 1:\n\nInput: init = 5, calls = ["incre...   
2  Example 1:\n\nInput: arr = [1,2,3], fn = funct...   
3  Example 1:\n\nInput: arr = [0,10,20,30], fn =

In [6]:
# Prompts with their explicit formatting
prompts = {
    "CODEGEN": lambda row: PROMPT_CODEGEN.format(
        leetcode_desc=row['Description'],
        schema=row['Function Header']
    ),
    "REFACTOR": lambda row: PROMPT_REFACTOR.format(
        leetcode_desc=row['Description'],
        gt_code=row['Solution'],
        schema=row['Function Header']
    ),
    "REFACTOR_V2": lambda row: PROMPT_REFACTOR_V2.format(
        gt_code=row['Solution'],
        schema=row['Function Header']
    ),
    "REFACTOR_V3": lambda row: PROMPT_REFACTOR_V3.format(
        leetcode_desc=row['Description'],
        examples=row['Examples'],
        constraint=row['Constraints'],
        gt_code=row['Solution'],
        schema=row['Function Header']
    ),
    "REFACTOR_V5": lambda row: PROMPT_REFACTOR_V5.format(
        leetcode_desc=row['Description'],
        gt_code=row['Solution'],
        schema=row['Function Header']
    )
}

In [7]:
models_with_delays = {
    "gemini-1.5-flash": 15,
    "gemini-1.5-pro": 10
}

In [8]:
import pandas as pd
import json
import time
import os
from tqdm import tqdm

# Max tokens for the response
MAX_TOKENS_ANSWER = 2000

# Ensure output directory exists
output_dir = "results-0.2"
os.makedirs(output_dir, exist_ok=True)

# Maximum number of retries
MAX_RETRIES = 5

# Wrap outer loop for models in tqdm
for model_name, delay in tqdm(models_with_delays.items(), desc="Models"):
    for prompt_name, prompt_formatter in tqdm(prompts.items(), desc=f"Prompts for {model_name}", leave=False):
        # Initialize lists to store results for this combination
        gt_code_lst = []
        lvl_lst = []
        ans = []
        rat = []

        # Wrap DataFrame iteration in tqdm for row-level progress
        for index, row in tqdm(df.iterrows(), desc=f"Examples for {prompt_name}", total=len(df), leave=False):
            retry_count = 0
            while retry_count < MAX_RETRIES:
                try:
                    # Generate the prompt using the specific formatter
                    filled_prompt = (
                        prompt_formatter(row)
                        + "\n"
                        + "Provide output in valid JSON. The data schema should be like this: "
                        + json.dumps(out_schema)
                    )

                    # Send the prompt to Gemini API
                    response_txt = gemini_json_response(
                        filled_prompt, gemini_schema, model_name, MAX_TOKENS_ANSWER
                    )
                    
                    # Parse the response
                    response_json = json.loads(response_txt[1:-1])

                    print("response_json: ")
                    print(response_json)

                    # Append results
                    gt_code_lst.append(row["Solution"])  # Ground truth code
                    lvl_lst.append(row["Difficulty"])  # Difficulty level
                    ans.append(response_json["code"])  # Refactored code
                    rat.append(response_json["rationale"])  # Rationale

                    # Break out of retry loop on success
                    break

                except Exception as e:
                    retry_count += 1
                    print(
                        f"Error with model {model_name}, prompt {prompt_name}, example {index} (Attempt {retry_count}): {str(e)}"
                    )
                    if retry_count >= MAX_RETRIES:
                        # Log error after exceeding retries
                        gt_code_lst.append(row["Solution"])  # Ground truth code
                        lvl_lst.append(row["Difficulty"])  # Difficulty level
                        ans.append("Error")
                        rat.append(str(e))
                        print(
                            f"Skipping example {index} after {MAX_RETRIES} attempts."
                        )

                # Delay before the next retry to prevent spamming the API
                time.sleep(delay)

            # Delay to prevent API rate limiting
            time.sleep(delay)

        # Create a DataFrame for the results
        output_df = pd.DataFrame()
        output_df["gt_code"] = gt_code_lst
        output_df["level"] = lvl_lst
        output_df["refactor_code"] = ans
        output_df["rationale"] = rat

        # Save results to a separate CSV file for each prompt-model combination
        output_file = os.path.join(
            output_dir, f"{model_name}_{prompt_name}_results.csv"
        )
        output_df.to_csv(output_file, index=False)
        print(f"Results saved to '{output_file}'")


Models:   0%|          | 0/1 [00:00<?, ?it/s]


response_json: 
{'code': '/**\n * @return {Function}\n */\nvar createHelloWorld = function() {\n    return () => "Hello World";\n};\n\n/**\n * const f = createHelloWorld();\n * f(); // "Hello World"\n */', 'rationale': "The original code was already quite efficient and readable. This refactoring makes it slightly more concise by using an arrow function.  Arrow functions have a more compact syntax and implicitly return the value if the function body is a single expression. This doesn't impact runtime significantly but does improve readability slightly by reducing verbosity.  The use of ...args in the original response was unnecessary and has been removed."}


response_json: 
{'code': '/**\n * @param {integer} init\n * @return { increment: Function, decrement: Function, reset: Function }\n */\nvar createCounter = function(init) {\n    let currentCount = init;\n\n    const increment = () => ++currentCount;\n    const decrement = () => --currentCount;    \n    const reset = () => (currentCount = init);\n\n    return { increment, decrement, reset };\n};', 'rationale': 'The provided code is already quite efficient and readable. This refactoring makes a few minor improvements:\n\n1. Uses const and arrow functions for conciseness where appropriate. \n2. Renames `presentCount` to `currentCount` for better clarity.\n3.  Maintains the core logic for incrementing, decrementing, and resetting the counter, ensuring functional equivalence with the original code.'}


response_json: 
{'code': '/**\n * @param {number[]} arr\n * @param {Function} fn\n * @return {number[]}\n */\nvar map = function(arr, fn) {\n    const transformedArr = [];\n    for (let i = 0; i < arr.length; i++) {\n        transformedArr[i] = fn(arr[i], i);\n    }\n    return transformedArr;\n};', 'rationale': 'This improved version uses a standard for loop which offers performance advantages, especially with larger arrays, compared to the forEach method.  By iterating through the array with a simple for loop, the code avoids the overhead associated with the callback function used in forEach. This change enhances both readability and run-time efficiency.'}


response_json: 
{'code': '/**\n * @param {number[]} arr\n * @param {Function} fn\n * @return {number[]}\n */\nvar filter = function(arr, fn) {\n    const filteredArr = [];\n    for (let i = 0; i < arr.length; i++) {\n        if (fn(arr[i], i)) {\n            filteredArr.push(arr[i]);\n        }\n    }\n    return filteredArr;\n};', 'rationale': 'This refactored code maintains the original logic while enhancing readability. The variable name is changed to filteredArr for better clarity. The core logic remains efficient, iterating through the array and applying the filter function.  No performance improvements were necessary as the original code already possessed optimal time complexity for this task.  Using a for loop over other methods is generally more performant in JavaScript.'}


...

response_json: 
{'code': "/**\n * @param {string} s\n * @param {string} p\n * @return {boolean}\n */\nvar isMatch = function(s, p) {\n    const m = s.length;\n    const n = p.length;\n\n    const dp = Array(m + 1).fill(false).map(() => Array(n + 1).fill(false));\n    dp[0][0] = true; // Empty string matches empty pattern\n\n    // Initialize first row (empty string)\n    for (let j = 1; j <= n; j++) {\n        if (p[j - 1] === '*') {\n            dp[0][j] = dp[0][j - 2];\n        }\n    }\n\n    for (let i = 1; i <= m; i++) {\n        for (let j = 1; j <= n; j++) {\n            if (p[j - 1] === '.' || p[j - 1] === s[i - 1]) {\n                dp[i][j] = dp[i - 1][j - 1];\n            } else if (p[j - 1] === '*') {\n                dp[i][j] = dp[0][j - 2];\n                if (p[j - 2] === '.' || p[j - 2] === s[i - 1]) {\n                    dp[i][j] = dp[i][j] || dp[i - 1][j];\n                }\n            }\n        }\n    }\n\n    return dp[m][n];\n};", 'rationale': "This refactore

response_json: 
{'code': '/**\n * @param {number[]} nums\n * @return {number}\n */\nvar maxCoins = function(nums) {\n    const n = nums.length;\n    nums.push(1);\n    nums.unshift(1);\n\n    const dp = Array(n + 2).fill(0).map(() => Array(n + 2).fill(0));\n\n    for (let len = 1; len <= n; len++) {\n        for (let i = 1; i <= n - len + 1; i++) {\n            const j = i + len - 1;\n            for (let k = i; k <= j; k++) {\n                dp[i][j] = Math.max(dp[i][j], nums[i - 1] * nums[k] * nums[j + 1] + dp[i][k - 1] + dp[k + 1][j]);\n            }\n        }\n    }\n\n    return dp[1][n];\n};', 'rationale': 'The provided code utilizes dynamic programming to determine the maximum coins obtainable by bursting balloons. To address boundary conditions, the input array `nums` is padded with 1s at both ends. A 2D array `dp` stores the maximum coins for subproblems. The code iterates through possible subarray lengths and starting points, calculating the maximum coins by considering eac

response_json: 
{'code': "/**\n * @param {character[][]} board\n * @return {void} Do not return anything, modify board in-place instead.\n */\nvar solveSudoku = function(board) {\n    const n = 9;\n    const isValid = (row, col, num) => {\n        for (let i = 0; i < n; i++) {\n            if (board[row][i] === num || board[i][col] === num) return false;\n        }\n        const startRow = 3 * Math.floor(row / 3);\n        const startCol = 3 * Math.floor(col / 3);\n        for (let i = 0; i < 3; i++) {\n            for (let j = 0; j < 3; j++) {\n                if (board[startRow + i][startCol + j] === num) return false;\n            }\n        }\n        return true;\n    };\n\n    const solve = () => {\n        for (let row = 0; row < n; row++) {\n            for (let col = 0; col < n; col++) {\n                if (board[row][col] === '.') {\n                    for (let num = 1; num <= 9; num++) {\n                        const numStr = num.toString();\n                        if (



Models: 100%|██████████| 1/1 [58:24<00:00, 3504.27s/it]

Results saved to 'results-0.2/gemini-1.5-pro_REFACTOR_V5_results.csv'
